In [115]:
# Source: https://gist.github.com/adrianespejo/5df28ce987db64ba753619502ee3d812

import json
import folium
import requests
import numpy as np
import geopandas as gpd

from streamlit_folium import st_folium


In [100]:
def nominatim_to_gdf(nominatim_query_url):
    """
    Function to convert nominatim queried url to geodataframe

    Parameters:
    ----------
        nominatim_query_url: str - url of the nominatim query result

    Returns:
    -------
        gdf: geodataframe - geodataframe of the query result
    """

    gdf = gpd.read_file(nominatim_query_url)
    return gdf


In [101]:
# converting the inputs into a json format
inputs = {"query": "Vietnam"}
response = requests.post("http://localhost:8000/processors/fetch_osm", data=json.dumps(inputs)).json()

print(response)


https://nominatim.openstreetmap.org/search?q=Vietnam&featureType=country&addresstype=country&namedetails=1&polygon_geojson=1&hierarchy=1&format=geojson


In [102]:
# test the query_result function
country_gdf = nominatim_to_gdf(response)
country_gdf.head()


,place_id,osm_type,osm_id,place_rank,category,type,importance,addresstype,name,display_name,namedetails,geometry
0,254250586,relation,49915,4,boundary,administrative,0.751195,country,Việt Nam,Việt Nam,"{'name': 'Việt Nam', 'name:af': 'Viëtnam', 'na...","MULTIPOLYGON (((114.55248 10.10177, 114.55268 ..."


In [103]:
country_gdf.to_crs(epsg=4326, inplace=True)
country_gdf.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [59]:
country_gdf["area"] = country_gdf.to_crs("+proj=cea").area
country_gdf["area"]

country_gdf["boundary"] = country_gdf.to_crs("+proj=cea").boundary.to_crs(country_gdf.crs)
country_gdf["boundary"]

country_gdf["centroid"] = country_gdf.to_crs("+proj=cea").centroid.to_crs(country_gdf.crs)
country_gdf["centroid"]


0    5.100199e+11
Name: area, dtype: float64

In [104]:
# check extend as array
country_extend = country_gdf.total_bounds
print(country_extend)


[102.1438643   7.6920852 114.8572578  23.3926918]


In [105]:
# Get the centroid of the country boundary
country_centroid = country_gdf.to_crs("+proj=cea").centroid.to_crs(country_gdf.crs)
country_centroid = country_centroid.total_bounds
print(country_centroid)

country_centroid = np.delete(country_centroid, [2, 3])
country_centroid[[0, 1]] = country_centroid[[1, 0]]
print(country_centroid)


[106.88531081  14.78677228 106.88531081  14.78677228]
[ 14.78677228 106.88531081]


In [106]:
m = folium.Map(location=country_centroid, zoom_start=6, tiles="CartoDB positron")

for _, r in country_gdf.iterrows():
    # without simplifying the representation of each borough, the map might not be displayed
    # sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    geo_j.add_to(m)

m


In [116]:
st_map = st_folium(m)
st_map


2023-10-27 10:37:10.952 
  command:

    streamlit run /home/nguyen/miniconda3/envs/geoenv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


{'last_clicked': None,
 'last_object_clicked': None,
 'last_object_clicked_tooltip': None,
 'last_object_clicked_popup': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': 7.6921062, 'lng': 102.1438643},
  '_northEast': {'lat': 23.3926672, 'lng': 114.8572578}},
 'zoom': 6,
 'last_circle_radius': None,
 'last_circle_polygon': None}